In [1]:
import boto3
import botocore
import json
import os
import numpy as np

In [2]:
def from_local_file(image_file_path):
    
    #get s3 url from event dictionary
    image_file = image_file_path
    # bucket = 'shoofping-demo'
    
    #check if file exists
    if not os.path.isfile(image_file):
        print("File not found!")
        return
    
    #pass it to the aws rekognition api
    r_client = boto3.client('rekognition')
    with open(image_file, 'rb') as image:
        response = r_client.detect_faces(Image={'Bytes': image.read()}, Attributes=['ALL'])
        # response = r_client.detect_faces(Image={'S3Object': {'Bucket': bucket , 'Name': file_name}}, Attributes=['ALL'])
    
    def get_landmark(landmark_type):
            for item in faceDetail['Landmarks']:
                if item['Type'] == landmark_type:
                    return item
        
    for faceDetail in response['FaceDetails']:
        print('The detected face is between ' + str(faceDetail['AgeRange']['Low']) 
              + ' and ' + str(faceDetail['AgeRange']['High']) + ' years old')
#         print('Here are the other attributes:')
#         print(json.dumps(faceDetail, indent=4, sort_keys=True))
        
        main_features = {}
        landmarks = {}
            
        leftPupil = get_landmark('leftPupil')["X"]
        rightPupil = get_landmark('rightPupil')["X"]
        main_features['pupillary_distance'] = abs(leftPupil - rightPupil)
        
        #calculate face shape
        forehead_w = abs(get_landmark('upperJawlineLeft')["X"] - get_landmark('upperJawlineRight')["X"]) 
        mid_w = abs(get_landmark('midJawlineLeft')["X"] - get_landmark('midJawlineRight')["X"]) 
        main_features['forehead_width'] = forehead_w
        main_features['midhead_width'] = mid_w
        
        mid_j_y = [get_landmark('midJawlineLeft')["Y"], get_landmark('midJawlineRight')["Y"]]
        mid_j_x = (get_landmark('midJawlineLeft')["X"], get_landmark('midJawlineRight')["X"])
        
        chin_angle = np.arctan2(mid_j_y, mid_j_x) * 180/np.pi
        jaw_angle = 180 - (chin_angle[0] + chin_angle[1])
        main_features['jaw_angle'] = jaw_angle
        main_features['f_w//mid_w'] = forehead_w/float(mid_w)
        main_features['abs(f_w-mid_w)'] = abs(forehead_w - mid_w)
        
        
        face_shape = "UNKNOWN"
        if forehead_w/(float(mid_w)) > 1.5 and jaw_angle < 90:
            face_shape = "HEART?"
        elif abs(forehead_w - mid_w) < 0.1 and jaw_angle > 70 and jaw_angle < 150:
            face_shape = "SQUARE?"
        elif abs(forehead_w - mid_w) < 1 and jaw_angle > 70:
            face_shape = "ROUND?"
            
        main_features['shape'] = face_shape
        
        print('Here are the calculated features:')
        print(json.dumps(main_features, indent=4, sort_keys=True))
        
        
        
  
        
        
            
        

    

In [3]:
from_local_file('C:/Temp/aws/faces/square.jpeg')

The detected face is between 26 and 43 years old
Here are the calculated features:
{
    "abs(f_w-mid_w)": 0.09383657574653625,
    "f_w//mid_w": 1.2454306309120873,
    "forehead_width": 0.47617098689079285,
    "jaw_angle": 83.45000785044746,
    "midhead_width": 0.3823344111442566,
    "pupillary_distance": 0.2134687900543213,
    "shape": "SQUARE?"
}
